<a href="https://colab.research.google.com/github/patelami3431/Web-Crawler/blob/master/HLT_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import urllib
from urllib import request
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from urllib.error import HTTPError
from nltk import word_tokenize
import math

In [2]:
def prepare_soup(url):
    try:
        p = request.urlopen(url)
        if(p.getcode() == 403):
            return False
        data = p.read()
        soup = BeautifulSoup(data, 'html.parser')
        return soup
    except HTTPError as e:
        return False


In [4]:
def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element.encode('utf-8'))):
        return False
    return True

In [5]:
def find_links(soup):
    s = []
    for link in soup.find_all('a'):
#             print(link.get('href'))
            s.append(link.get('href'))
    return s

In [6]:
def all_text(soup):
    # kill all script and style elements
    for script in soup(['style', 'script', '[document]', 'head', 'title']):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = text.splitlines()
    # break multi-headlines into a line each
    chunks = (phrase for line in lines for phrase in line.split(" "))
    # drop blank lines
    text = ' '.join(chunk for chunk in chunks if chunk)
    return text

In [62]:
def sent_tokenizer(filename):
    file = open(filename,"r")
    content = file.read()
    content = content.lower()
    content = ''.join([i for i in content if not i.isdigit()])
    punctuations = '''!()-[]{};:'"\,<>/?@#$%^&*_~'''
    for x in content: 
        if x in punctuations: 
            content = content.replace(x, " ") 
    content = re.sub(r'[^a-zA-Z .]+', ' ',content)
    sents = sent_tokenize(content)
    with open('tokenized_'+filename, 'w+') as f:
        for s in sents:
            if "retrieved" in s or "register" in s or "reply" in s:
                continue
            else:
                f.write(s+'\n')
    f.close()

In [48]:
def create_tf_idf_dict(text, num_docs):
    tf_dict = {}
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if w.isalpha()and w not in stop_words]
    for t in tokens:
        if t in tf_dict:
            tf_dict[t] += 1
        else:
            tf_dict[t] = 1
    
    for t in tf_dict.keys():
        tf_dict[t] = tf_dict[t] / len(tokens)
        
    
    vocab = set(tokens)
    idf_dict = {}
    for term in vocab:
        temp = ['x' for voc in vocab if term in voc]
        idf_dict[term] = math.log((1+num_docs) / (1+len(temp)))
        
    tf_idf = {}
    for t in tf_dict.keys():
        tf_idf[t] = tf_dict[t] * idf_dict[t]
        
    return tf_idf

In [60]:
def web_crawler():
    soup = prepare_soup("https://en.wikipedia.org/wiki/List_of_diets")
    s = find_links(soup)
    counter = 0
    final = []
    crawl = True
    i = 0
    while (crawl):
        for link in s:
            if link and ('diet' in link or 'Diet' in link)  and link.startswith('http'):
                final.append(link)
                counter += 1
        if (counter > 15):
            crawl = False
            break
        else:
            soup1 = prepare_soup(final[i])
            while soup1 is False:
                i += 1
                soup1 = prepare_soup(final[i])
                s = find_links(soup1)
                continue
            i += 1
            s = find_links(soup1)                
            

#     print(counter)
    
    file = "file_"
    j = 0
    files = []
    for url in final:
        soup2 = prepare_soup(url)
        if soup2 is False:
            continue
        text = all_text(soup2)
        filename = file + str(j) + ".txt"
        if text:
            with open(filename, "w+") as f:
                f.write(text)
            files.append(filename)
            f.close()
            j += 1
    print("The total number of files:",j) 
    vocab = []
    text = ''
    for f in files:
        sent_tokenizer(f)
    
    for f in files:
        with open("tokenized_" + f, "r") as file:
            text += file.read() + " "
        file.close()
            
    text = text.lower()
    text = re.sub(r'[.?!,:;()\-]', ' ', text)
    
    tf_idf_dict = create_tf_idf_dict(text, len(files))
    
    l = 0
    for k in sorted(tf_idf_dict, key=lambda k:tf_idf_dict[k], reverse= True):
        if l < 35:
            print (k,"->",tf_idf_dict[k])
            l += 1
    

In [63]:
web_crawler()

The total number of files: 19
weight -> 0.009885627818970769
jpg -> 0.007842513715181324
calorie -> 0.006746582465659783
diets -> 0.006573154804484198
foods -> 0.005763207211267264
loss -> 0.005346589822500956
people -> 0.0053212481419288435
also -> 0.005305229866152072
eating -> 0.0052398429405568
fasting -> 0.004917391067291766
much -> 0.004843905529964935
may -> 0.004836778098975508
atkins -> 0.004721663739351493
atkin -> 0.0043419004213106635
kb -> 0.004305046350585185
kidney -> 0.004166173887663082
blood -> 0.0040365879416374454
cancer -> 0.0040365879416374454
body -> 0.0040306484158129235
calories -> 0.0038059257735438773
protein -> 0.003610846953451715
based -> 0.0035158246652029854
many -> 0.0032307578004567975
pounds -> 0.003229270353309957
disease -> 0.0032245187326503384
lose -> 0.0030749551232440967
food -> 0.0030639205652456055
medicine -> 0.003040713223959339
gluten -> 0.0029986081852163885
png -> 0.002883277101169604
dietary -> 0.0028506686474618803
high -> 0.00278690571